In [15]:
# NOTE: this notebook requires the following, which is not by default installed in this package
! pip install miceforest

  Using cached miceforest-6.0.3-py3-none-any.whl.metadata (35 kB)
  Using cached lightgbm-4.5.0.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Using cached miceforest-6.0.3-py3-none-any.whl (40 kB)
  Created wheel for lightgbm: filename=lightgbm-4.5.0-py3-none-linux_x86_64.whl size=2740297 sha256=bd4a608bac9bc9c245f901d153e4b76082d82da96ac96ed7a1fcc8c8a7b86856
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7c/16/bf/61bc6295609c0752bc0f5b774a41f3413bca3afe3a73d2e47d
Successfully built lightgbm


In [1]:
import pandas as pd
import numpy as np

In [2]:
df_2018 = pd.read_csv("survey_clean_2018_raw.csv")

### Pre-process, extract as much as possible from 2018

In [3]:
# NOTE: 2018 and 2023 differ pretty significantly in column names and structure.
# so, we map the 2018 columns to 2023 columns as best we can, and impute at the end
# using 2023 as ground truth

df_mapped = pd.DataFrame()

family_cols_2018 = [
    "Paid Family Leave  (e.g. Parental Leave, Bereavement Leave)",
    "Child Care (e.g. referrals, reimbursement, lactation room, and/or services)",
    "Elder Care (e.g. referrals, reimbursement, and/or services)",
    "Adoption/Infertility Support (financial)",
    "Emergency Leave/Life Events Time Off (e.g. military service, bereavement)"
]

def any_family_commitments(row):
    for col in family_cols_2018:
        if row[col] == 1.0:
            return 1
    return 0

df_mapped["family_commitments"] = df_2018.apply(any_family_commitments, axis=1)

cult_col_2018 = "Recognition of Religious Diversity (e.g. extended holidays, dedicated prayer room)"

df_mapped["cultural_religious_practices"] = df_2018[cult_col_2018].apply(
    lambda x: 1 if x == 1.0 else 0
)

flex_cols_2018 = [
    "Flex Time/Alternative Schedule",
    "Job Share",
    "Temporary or Permanent Switch to Part Time",
    "Telecommute/Remote Workplace"
]

def any_flex_arrangements(row):
    for col in flex_cols_2018:
        if row[col] == 1.0:
            return 1
    return 0

df_mapped["flexible_working_arrangements"] = df_2018.apply(any_flex_arrangements, axis=1)

df_mapped["recognition_from_colleagues_peers"] = df_2018["Recognition from Colleagues/Peers"]

df_mapped["leadership_opportunities"] = df_2018["Leadership Opportunities"]

df_mapped["socialize_with_peers_within_company"] = df_2018["Socializing With Peers Within Your Company"]

mgr_expect_col_2018 = "I let my manager know what I want/expect - you don't get what you don't ask for!"

df_mapped["i_let_manager_know"] = df_2018[mgr_expect_col_2018]

pursue_col_2018 = "I don't pursue opportunities unless I fulfill all/most of the requirements on a job description"

def invert_pursue(val):
    if pd.isna(val):
        return np.nan 
    
    val_str = str(val).lower().strip()
    
    if val_str == "strongly disagree":
        return 5
    elif val_str == "disagree":
        return 4
    elif val_str == "neither agree nor disagree":
        return 3
    elif val_str == "agree":
        return 2
    elif val_str == "strongly agree":
        return 1
    else:
        return 0

df_mapped["i_pursue_opps"] = df_2018[pursue_col_2018].apply(invert_pursue)

df_mapped["i_am_paid_fairly"] = df_2018["I am being paid fairly for the work that I do"]

df_mapped["i_avoid_salary_discussions"] = df_2018["I avoid discussions about salary/incentives"]

df_mapped["primary_caregiver"] = df_2018["Primary Caregiver?"]

df_mapped["organization_type"] = df_2018["Organization type"]

df_mapped["size_of_org"] = df_2018["Size of Organization"]

df_mapped["length_with_employer"] = df_2018["Length with current employer"]

print(df_mapped.head())

   family_commitments  cultural_religious_practices  \
0                   1                             0   
1                   1                             0   
2                   1                             0   
3                   1                             0   
4                   1                             0   

   flexible_working_arrangements   recognition_from_colleagues_peers  \
0                              1                      Very Satisfied   
1                              1  Neither Dissatisfied nor Satisfied   
2                              1                  Somewhat Satisfied   
3                              1                      Very Satisfied   
4                              1                                 NaN   

             leadership_opportunities  socialize_with_peers_within_company  \
0  Neither Dissatisfied nor Satisfied                                  1.0   
1                   Very Dissatisfied                                  1.0   
2 

In [4]:
mapping = {
    "how_long_have_you_worked_for_your_current_employer": "length_with_employer",
    "which_of_the_following_groups_best_describes_your_employment_level": None,
    "which_of_the_following_groups_best_describes_the_organization_that_you_work_for": "organization_type",
    "flexible_working_arrangements": "flexible_working_arrangements",
    "culturalreligious_practices": "cultural_religious_practices",
    "approximately_what_is_the_size_of_your_employer's_workforce": "size_of_org",
    "i_believe_i_am_being_paid_fairly_for_the_work_that_i_do": "i_am_paid_fairly",
    "family_commitments": "family_commitments",
    "yes_one_or_more_children": "primary_caregiver",
    "leadership_opportunities": "leadership_opportunities",
    "black": None,
    "white": None,
    "asian": None,
    "latinaeox": None,
    "recognition_from_colleaguespeers": "recognition_from_colleagues_peers",
    "what_country_do_you_currently_live_in": None,
    "is_your_first_language_the_primary_language_of_your_workplace_": None,
    "socialize_with_peers_within_my_company": "socialize_with_peers_within_company",
    "i_pursue_job_opportunities_even_if_i_do_not_fulfill_all_of_the_requirements_on_a_job_description": "i_pursue_opps",
    "i_avoid_discussions_about_salary_and_incentives": "i_avoid_salary_discussions",
    "i_advocate_for_myself_when_appropriate_so_that_my_manager_knows_what_i_want_and_expect": None,
    "i_trust_that_i_am_compensated_appropriately_for_my_accomplishments": None,
    "i_let_my_manager_know_what_i_want_with_the_expectation_that_it_will_generate_growth_opportunities": "i_let_manager_know"
}
# invert mapping
mapping = {v: k for k, v in mapping.items()}

df_mapped.rename(columns=mapping, inplace=True)

In [5]:
for col in df_mapped.columns:
    print(f"{col}: {df_mapped[col].unique()}")

family_commitments: [1 0]
culturalreligious_practices: [0 1]
flexible_working_arrangements: [1 0]
recognition_from_colleaguespeers: ['Very Satisfied' 'Neither Dissatisfied nor Satisfied'
 'Somewhat Satisfied' nan 'Somewhat Dissatisfied' 'Very Dissatisfied'
 'Not Sure']
leadership_opportunities: ['Neither Dissatisfied nor Satisfied' 'Very Dissatisfied'
 'Somewhat Dissatisfied' nan 'Somewhat Satisfied' 'Very Satisfied'
 'Not Sure']
socialize_with_peers_within_my_company: [ 1. nan]
i_let_my_manager_know_what_i_want_with_the_expectation_that_it_will_generate_growth_opportunities: [nan 'Neither Agree nor Disagree' 'Agree' 'Disagree' 'Strongly Agree'
 'Strongly Disagree']
i_pursue_job_opportunities_even_if_i_do_not_fulfill_all_of_the_requirements_on_a_job_description: [nan  4.  2.  1.  3.  5.]
i_believe_i_am_being_paid_fairly_for_the_work_that_i_do: [nan 'Agree' 'Disagree' 'Strongly Disagree' 'Neither Agree nor Disagree'
 'Strongly Agree']
i_avoid_discussions_about_salary_and_incentives: [na

In [9]:

yes_no_map = {
    "Yes": 1,
    "No": 0,
    "Share Equal Responsibility": 1,
    "Not Applicable": 0,
    np.nan: 0,
    None: 0
}

sat_dsat_map = {
    "Very Dissatisfied": 1,
    "Somewhat Dissatisfied": 2,
    "Dissatisfied": 2, 
    "Neither Dissatisfied nor Satisfied": 3,
    "Not Sure": 3,
    "Not sure": 3,  
    "Somewhat Satisfied": 4,
    "Satisfied": 4, 
    "Very Satisfied": 5,
    np.nan: 0,
    None: 0
}

agree_map = {
    "Strongly Disagree": 1,
    "Disagree": 2,
    "Neither Agree nor Disagree": 3,
    "Neither": 3,
    "Not sure": 3,
    "Agree": 4,
    "Strongly Agree": 5,
    np.nan: 0,
    None: 0
}

how_long_map = {
    "< 1 year": 0,
    "1-5 years": 0,
    "6 – 10 years": 0,
    "11 – 20 years": 1,
    "> 21 years": 1,
    np.nan: -1, 
    None: -1
}

org_map = {
    "Academic Institution/Library": 1,
    "Consulting (serving publishing)": 2,
    "Foundation": 3,  
    "Industry Organization/Trade Association": 4,
    "Non-profit organization or association": 5, 
    "Not listed (please specify)": 6,
    "Publisher (Commercial, Society, and University)": 7,
    "Publishing Services Provider": 8,
    "Technology/Analytics": 9,
    np.nan: 0,
    None: 0
}

size_map = {
    "1": 1,
    "2-10": 2,
    "11-100": 3,
    "101-500": 4,
    "501 -2,000": 5,  
    "501-2,000": 5,
    "2,001- 10,000": 6,
    "2001-10,000": 6, 
    "10,001- 20,001+": 7,
    "10,001-20,001+": 7,
    np.nan: 0,
    None: 0
}

column_mapping = {
    "family_commitments": yes_no_map,
    "culturalreligious_practices": yes_no_map,
    "flexible_working_arrangements": yes_no_map,
    
    "socialize_with_peers_within_my_company": yes_no_map,
    "yes_one_or_more_children": yes_no_map,
    
    "recognition_from_colleaguespeers": sat_dsat_map,
    "leadership_opportunities": sat_dsat_map,
    
    "i_let_my_manager_know_what_i_want_with_the_expectation_that_it_will_generate_growth_opportunities": agree_map,
    "i_pursue_job_opportunities_even_if_i_do_not_fulfill_all_of_the_requirements_on_a_job_description": agree_map,
    "i_believe_i_am_being_paid_fairly_for_the_work_that_i_do": agree_map,
    "i_avoid_discussions_about_salary_and_incentives": agree_map,
    
    "which_of_the_following_groups_best_describes_the_organization_that_you_work_for": org_map,
    "approximately_what_is_the_size_of_your_employer's_workforce": size_map,
    "how_long_have_you_worked_for_your_current_employer": how_long_map,
}

def map_column(series: pd.Series, mapping: dict):
    def convert_value(val):
        if val in mapping:
            return mapping[val]
        if isinstance(val, (int, float)) and not np.isnan(val):
            return val
        return 0
    
    return series.apply(convert_value)

for col, col_map in column_mapping.items():
    if col in df_mapped.columns:
        df_mapped[col] = map_column(df_mapped[col], col_map)
        df_mapped[col] = df_mapped[col].astype(int)

# # for how_long_have_you_worked_for_your_current_employer, ignore if value is -1
df_mapped = df_mapped[df_mapped["how_long_have_you_worked_for_your_current_employer"] != -1]


In [11]:
# load 2023 data
df_2023 = pd.read_csv("survey_2023.csv")

# compare max and min of each column
for col in df_mapped.columns:
    if col in df_2023.columns:
        print(f"{col}: 2018 min: {df_mapped[col].min()}, 2018 max: {df_mapped[col].max()}")
        print(f"{col}: 2023 min: {df_2023[col].min()}, 2023 max: {df_2023[col].max()}")

family_commitments: 2018 min: 0, 2018 max: 1
family_commitments: 2023 min: 0, 2023 max: 1
culturalreligious_practices: 2018 min: 0, 2018 max: 1
culturalreligious_practices: 2023 min: 0, 2023 max: 1
flexible_working_arrangements: 2018 min: 0, 2018 max: 1
flexible_working_arrangements: 2023 min: 0, 2023 max: 1
recognition_from_colleaguespeers: 2018 min: 0, 2018 max: 5
recognition_from_colleaguespeers: 2023 min: 0, 2023 max: 5
leadership_opportunities: 2018 min: 0, 2018 max: 5
leadership_opportunities: 2023 min: 0, 2023 max: 5
socialize_with_peers_within_my_company: 2018 min: 0, 2018 max: 1
socialize_with_peers_within_my_company: 2023 min: 0, 2023 max: 1
i_let_my_manager_know_what_i_want_with_the_expectation_that_it_will_generate_growth_opportunities: 2018 min: 0, 2018 max: 5
i_let_my_manager_know_what_i_want_with_the_expectation_that_it_will_generate_growth_opportunities: 2023 min: 0, 2023 max: 5
i_pursue_job_opportunities_even_if_i_do_not_fulfill_all_of_the_requirements_on_a_job_descrip

### Impute 
Use 2023 distribution to fill in the missing columns with standard MICE imputation.

In [16]:
df_2018 = df_mapped.copy()

In [18]:
import miceforest as mf

In [ ]:
df_2018_extended = df_2018.reindex(columns=df_2023.columns)
df_combined = pd.concat([df_2023, df_2018_extended], ignore_index=True)

kernel = mf.ImputationKernel(
    data=df_combined,
    num_datasets=1,
    random_state=42
)

kernel.mice(5)

df_imputed = kernel.complete_data(dataset=0, inplace=False)

num_rows_2023 = df_2023.shape[0]

df_2023_imputed = df_imputed.iloc[:num_rows_2023].reset_index(drop=True)
df_2018_imputed = df_imputed.iloc[num_rows_2023:].reset_index(drop=True)

In [24]:
# make df_2018_imputed only integer values
for col in df_2018_imputed.columns:
    df_2018_imputed[col] = df_2018_imputed[col].astype(int)

df_2018_imputed.to_csv("survey_2018.csv", index=False)

In [27]:
# compare max and min of each column
for col in df_2018_imputed.columns:
    print(f"{col}: 2018 min: {df_2018_imputed[col].min()}, 2018 max: {df_2018_imputed[col].max()}")
    print(f"{col}: 2023 min: {df_2023[col].min()}, 2023 max: {df_2023[col].max()}")
    print()
    print(f"{col}: 2018 mean: {df_2018_imputed[col].mean()}, 2018 std: {df_2018_imputed[col].std()}")
    print(f"{col}: 2023 mean: {df_2023[col].mean()}, 2023 std: {df_2023[col].std()}")
    print()

how_long_have_you_worked_for_your_current_employer: 2018 min: 0, 2018 max: 1
how_long_have_you_worked_for_your_current_employer: 2023 min: 0, 2023 max: 1

how_long_have_you_worked_for_your_current_employer: 2018 mean: 0.27837514934289126, 2018 std: 0.4484670729874769
how_long_have_you_worked_for_your_current_employer: 2023 mean: 0.3192857142857143, 2023 std: 0.4663664894056162

which_of_the_following_groups_best_describes_your_employment_level: 2018 min: 1, 2018 max: 5
which_of_the_following_groups_best_describes_your_employment_level: 2023 min: 0, 2023 max: 5

which_of_the_following_groups_best_describes_your_employment_level: 2018 mean: 2.8410991636798086, 2018 std: 0.8610846259095377
which_of_the_following_groups_best_describes_your_employment_level: 2023 mean: 2.7657142857142856, 2023 std: 0.946440714217451

which_of_the_following_groups_best_describes_the_organization_that_you_work_for: 2018 min: 0, 2018 max: 9
which_of_the_following_groups_best_describes_the_organization_that_you